In [2]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np

cuda:0


In [22]:
import torch
import torch.nn as nn

# 定义一个简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 第一个全连接层
        self.relu = nn.ReLU()  # ReLU激活函数
        self.fc2 = nn.Linear(hidden_size, output_size)  # 第二个全连接层
        self.relu2 = nn.ReLU()  # ReLU激活函数
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out

# 定义输入、隐藏层和输出的维度
input_size = 10
hidden_size = 20
output_size = 10

# 创建一个SimpleNN模型
model = SimpleNN(input_size, hidden_size, output_size)

# 打印模型的结构
print(model)


SimpleNN(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (relu2): ReLU()
)


In [23]:
class customloss(nn.Module):
    def __init__(self, theta):
        super(customloss, self).__init__()
        self.theta = theta
    def forward(self, output_list, y_list):
        theta = torch.tensor(self.theta, dtype= torch.float32).to(device) 
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)))  
            loss += torch.sum(torch.log(prob) * y)  
            index += 1  
        return -loss 

In [ ]:
feature_list = []
purchase_list = []

for t in range(0,500):
    context = CONTEXT_ARRAY[t]
    
    